### ML notebook

In [1]:
n_samples = 10000
gens_per_ideal = 2

In [2]:
# P.<x,y> = PolynomialRing(QQ, 2, order='deglex')
P.<x,y> = PolynomialRing(QQ, 2, order='lex')
# P.<x,y> = PolynomialRing(QQ, 2, order='degrevlex')
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('wdegrevlex',(1,3)))
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('negwdegrevlex',(1,3)))

#### Functions to create features and label

In [3]:
def is_GB_worth(GB): # Receives a GB
    n_depend1=0
    n_depend2=0
    for pol in GB:
        depend_x=False
        depend_y=False
        
        # Comprobamos si depende de x y de y
        if pol.degree(x) > 0:
            depend_x=True
        if pol.degree(y) > 0:
            depend_y=True
            
        # Actualizamos el numero de de polinomios que dependen de cada cosa
        if depend_x and depend_y:
            n_depend2 += 1
        elif depend_x or depend_y:
            n_depend1 += 1
    if n_depend1 >= n_depend2:
        return True
    else:
        return False
    
def terms_on_x(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[0] > 0:
                count+=1
    return count
        
def terms_on_y(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[1] > 0:
                count+=1
    return count

In [4]:
from collections import defaultdict

def n_homogeneous_comps(pol):
    dic = defaultdict(pol.parent())
    
    for coeff,monom in pol:
        dic[monom.degree()] += coeff * monom
        
    return len(dic)

#### Voy a generar una lista de ideales para los polinomios generados aleatoriamente y con información adicional

In [5]:
def do_experiment():
    lstOfIdeals = []
    isWorthCount = 0
    for i in range(n_samples):
        generators = []
        n_homogeneous = 0
        total_homogeneous_comps=0

        for j in range(gens_per_ideal):
            p = P.random_element(degree=5)

            # homogeneous components for the given polynomial
            total_homogeneous_comps += n_homogeneous_comps(p)

            generators.append(p)
            if p.is_homogeneous():
                n_homogeneous += 1

        # Definimos el ideal a partir de los generadores anteriores
        I = ideal(generators)
        B = I.groebner_basis()
        isWorth = is_GB_worth(B)
        
        if isWorth:
            isWorthCount+=1
        
        lstOfIdeals.append([I, 
                            n_homogeneous, 
                            terms_on_x(generators), 
                            terms_on_y(generators), 
                            terms_on_x(generators) - terms_on_y(generators),
                            total_homogeneous_comps, 
                            isWorth])
        
    print("Worth it bases in the given dataset "+str(isWorthCount))
        
    import numpy as np

    X = np.array([[item[1],item[2],item[3],item[4],item[5]] for item in lstOfIdeals])
    y = np.array([item[6] for item in lstOfIdeals])

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    from sklearn import svm
    
    classifier = svm.SVC(C= 4096, class_weight= None, gamma= 1/8192, kernel= 'rbf')
    
    classifier.fit(X_train, y_train)
    
    preds = classifier.predict(X_test)
    
    hits=0
    fails=0
    for i in range(len(y_test)):
        if y_test[i] == preds[i]:
            hits+=1
        else:
            fails+=1
    
    return hits/(hits+fails) # Return score (between 0 and 1)

In [6]:
iters=2
t_score=0.0

for i in range(iters):
    print('INFO: iter '+str(i+1))
    t_score += do_experiment()
    
avg_score=t_score/iters

INFO: iter 1
Worth it bases in the given dataset 5431
INFO: iter 2
Worth it bases in the given dataset 5479


In [7]:
avg_score

0.634848484848485